In [1]:
import pandas as pd
import numpy as np

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# будем отображать графики прямо в jupyter'e
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 

#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5
df_deaths=pd.read_csv('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_deaths_global.csv&filename=time_series_covid19_deaths_global.csv')
df=pd.read_csv('https://data.humdata.org/hxlproxy/api/data-preview.csv?url=https%3A%2F%2Fraw.githubusercontent.com%2FCSSEGISandData%2FCOVID-19%2Fmaster%2Fcsse_covid_19_data%2Fcsse_covid_19_time_series%2Ftime_series_covid19_confirmed_global.csv&filename=time_series_covid19_confirmed_global.csv')

In [2]:
place_df=df.iloc[:,:4] #место
days_df=df.iloc[:,4:] #абсолютные значения по дням
delta_df=days_df-days_df.shift(periods=1, axis=1) #изменения по дням
Country='Russia' #страна

In [3]:
increase_df=pd.concat((place_df,delta_df),axis=1)

In [4]:
total_df=pd.concat((place_df,days_df),axis=1)

In [5]:
total_Country=total_df[total_df['Country/Region']==Country]
total_Country_deaths=df_deaths[df_deaths['Country/Region']==Country]

In [6]:
increase_Country=increase_df[increase_df['Country/Region']==Country]

In [7]:
new_tr=total_Country.iloc[:,4:].T
new_tr.columns=['Total']

In [8]:
new_ir=increase_Country.iloc[:,4:].T
new_ir.columns=['Increase']

In [9]:
new_dr=total_Country_deaths.iloc[:,4:].T
new_dr.columns=['Deaths']

In [10]:
df_Country=pd.concat((new_tr,new_ir,new_dr),axis=1)

In [11]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [12]:
trace0 = go.Scatter(
    x=df_Country.index,
    y=df_Country.Total,
    name='Total'
)
trace1 = go.Scatter(
    x=df_Country.index,
    y=df_Country.Increase,
    name='Increase'
)
trace2 = go.Scatter(
    x=df_Country.index,
    y=df_Country.Deaths,
    name='Deaths'
)
# определяем массив данных и задаем title графика в layout
data = [trace0,trace1,trace2]
layout = {'title': 'Coronavirus statistics in '+Country}

# cоздаем объект Figure и визуализируем его
fig = go.Figure(data=data, layout=layout)
iplot(fig, show_link=False)
plotly.offline.plot(fig, filename='Coronavirus statistics in '+Country, show_link=False)

'Coronavirus statistics in Russia.html'

In [13]:
df_last_world=pd.concat((df['Country/Region']+' '+df['Province/State'].replace(np.nan,''),
                         df.iloc[:,-1],df_deaths.iloc[:,-1],
                         df['Lat'],df['Long']),axis=1)
df_last_world.columns=['Country','Cases','Deaths','Lat','Long']
df_last_world.head()
df_last_world.Cases[df_last_world.Cases < 0] = 0
df_last_world.Deaths[df_last_world.Deaths < 0] = 0
df_last_world=df_last_world.sort_values(by='Cases',ascending=False)
#df_last_world=df_last_world.groupby(['Country']).sum()

In [14]:
df_last_world.head(20)

,Country,Cases,Deaths,Lat,Long
225,US,965785,54881,37.0902,-95.7129
201,Spain,226629,23190,40.0000,-4.0000
137,Italy,197675,26644,43.0000,12.0000
116,France,160847,22856,46.2276,2.2137
120,Germany,157770,5976,51.0000,9.0000
223,United Kingdom,152840,20732,55.3781,-3.4360
213,Turkey,110130,2805,38.9637,35.2433
133,Iran,90481,5710,32.0000,53.0000
187,Russia,80949,747,60.0000,90.0000
62,China Hubei,68128,4512,30.9756,112.2707


In [15]:
tc=len(df_last_world)
limits=[(0,int(tc/20)),(int(tc/20+1),int(tc/10)),(int(tc/10+1),int(tc/3)),(int(tc/3+1),int(tc*2/3)),(int(tc*2/3+1),int(tc))]

In [16]:
df_last_world['text'] = df_last_world['Country'] + '<br>Infected ' + (df_last_world['Cases']).astype(str)+'<br>Deaths ' + (df_last_world['Deaths']).astype(str)

colors = ["purple","darkred","orange","pink","blue","magenta"]
scale = 200
fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df_last_world[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = df_sub['Long'],
        lat = df_sub['Lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Cases']/scale,
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1])))
fig.update_geos(
    projection_type="natural earth",
    visible=False, resolution=50,
    showcountries=True, countrycolor="RebeccaPurple",
    showland=True, landcolor="LightGreen",
    showocean=True, oceancolor="LightBlue",
    showlakes=True, lakecolor="Blue",
)
fig.update_layout(
        title_text = 'World coronavirus',
        showlegend = True,
    )

#fig.show()
plotly.offline.plot(fig, filename='Coronavirus in the world', show_link=False)

'Coronavirus in the world.html'